# Worksheet 17

Name:  Patrick Meade

UID:   U31683289

### Topics

- Recommender Systems

### Recommender Systems

In the example in class of recommending movies to users we used the movie rating as a measure of similarity between users and movies and thus the predicted rating for a user is a proxy for how highly a movie should be recommended. So the higher the predicted rating for a user, the higher a recommendation it would be.

a) Consider a streaming platform that only has "like" or "dislike" (no 1-5 rating). Describe how you would build a recommender system in this case.

The recommender system that I would build would either be based on a ratio of like:dislike/total amount of likes and dislikes. Through that I would also track the overal timeline of the likes and dislikes depending on the amount of likes and dislikes in a given period can increase or decrease the recomendation

b) Describe 3 challenges of building a recommender system

1. The data and the quality of data
2. There be next to none starting data to begin with.
3. You may not know what column of data is not being 

c) Why is SVD not an option for collaborative filtering?

SVD is not an option due to the possibility of the data not being completely there or there isn't enough of a sample size for the SVD to accurate do collaborative filtering

d) Use the code below to train a recommender system on a dataset of amazon movies

In [3]:
import findspark
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

findspark.init()
conf = SparkConf().set("spark.executor.memory", "28g").set("spark.driver.memory", "28g").set("spark.driver.cores", "8")
sc = SparkContext.getOrCreate(conf)
spark = SparkSession.builder.getOrCreate()

init_df = pd.read_csv("./train.csv").dropna()
init_df['UserId_fact'] = init_df['UserId'].astype('category').cat.codes
init_df['ProductId_fact'] = init_df['ProductId'].astype('category').cat.codes

from sklearn.model_selection import train_test_split

X_train_processed, X_test_processed, Y_train, Y_test = train_test_split(
    init_df.drop(['Score'], axis=1),
    init_df['Score'],
    test_size=1/4.0,
    random_state=0
)

X_train_processed['Score'] = Y_train

df = spark.createDataFrame(X_train_processed[['UserId_fact', 'ProductId_fact', 'Score']])
als = ALS(userCol="UserId_fact", itemCol="ProductId_fact", ratingCol="Score", coldStartStrategy="drop", nonnegative=True)

param_grid = ParamGridBuilder()\
    .addGrid(als.rank, [50, 100])\
    .addGrid(als.regParam, [0.05, 0.1])\
    .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="Score")
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)
rec_sys = cv.fit(df).bestModel


mean_score = Y_train.mean()
rec['prediction'].fillna(mean_score, inplace=True)

if len(rec) != len(X_test_processed):
    print("Warning: Mismatched number of predictions and test instances.")

X_test_processed['predicted_Score'] = rec['prediction']

if X_test_processed['predicted_Score'].isna().any() or Y_test.isna().any():
    print("Error: NaN values found in predictions or actual scores.")

    X_test_processed['predicted_Score'].fillna(mean_score, inplace=True)

try:
    rmse = mean_squared_error(Y_test, X_test_processed['predicted_Score'], squared=False)
    print("Kaggle RMSE = ", rmse)
except ValueError as e:
    print("Error during RMSE calculation:", e)


Error: NaN values found in predictions or actual scores.
Kaggle RMSE =  1.2601625497985345


/var/folders/17/0r4lptpx477gnn898k7b6c3r0000gn/T/ipykernel_38344/114446703.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rec['prediction'].fillna(mean_score, inplace=True)
/var/folders/17/0r4lptpx477gnn898k7b6c3r0000gn/T/ipykernel_38344/114446703.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al